<a href="https://colab.research.google.com/github/KimberlySalazarB/Proyecto_Final_Comunicacion/blob/main/Proyecto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1
!pip install catboost

# **Importación de los modulos y bibliotecas necesarios para el entrenamiento.**

In [ ]:
#2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

Se carga la base de datos desde la URL a un DataFrame, y las columnas se nombran se nombra segun la lista 'colum_naes'

In [ ]:
#3
# Descargar el conjunto de datos
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
           'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
           'hours-per-week', 'native-country', 'income']
data = pd.read_csv(url, names=columns, na_values=' ?')

In [ ]:
#4
# Descripción del conjunto de datos
print("Descripción del conjunto de datos:")
print(data.head())

Descripción del conjunto de datos:
   age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital-status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  income  
0          2174             0    

In [ ]:
#5
data.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [ ]:
#6
# Número y tipo de características
print("Número y tipo de características:")
print(data.dtypes)

Número y tipo de características:
age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object


En seccion se realiza el procesamiento de datos lo cual se se identifica las caracteristicas categoricas en los dato y creamos un transformador de columnas lo cual codifica estas caracteristicas.

# **Metodología**

In [ ]:
#7
# Verificar qué columnas tienen datos faltantes
fal_data = data.isnull().sum()
print(fal_data)

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64


In [ ]:
#8
# Rellenar los datos faltantes con la palabra "desconocido"
data = data.fillna("desconocido")
# Verificar qué columnas tienen datos faltantes después de la eliminación
missing_data = data.isnull().sum()
print(missing_data)

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64


Aqui separamos los datos en conjunto de caracterisiticas que es 'x' y mas el vector de objetivos 'y'. Luego se didvide estos conjunto de datos en entrenamiento.

In [ ]:
#9
# Separar características (X) y variable objetivo (y)
X = data.drop('income', axis=1)
y = data['income']
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#10
# Número de muestras en los conjuntos de entrenamiento y prueba
print("Número de muestras de entrenamiento:", len(X_train))
print("Número de muestras de prueba:", len(X_test))
print("Número de muestras por clase:")
print(y_train.value_counts())

Número de muestras de entrenamiento: 26048
Número de muestras de prueba: 6513
Número de muestras por clase:
 <=50K    19778
 >50K      6270
Name: income, dtype: int64


*Procesamiento de datos*

# Selección y extracción de características

In [ ]:
#11
### Procesamiento de datos
# Selección y extracción de características
# Columnas numéricas: aplicar escalado estándar
# Columnas categóricas: aplicar codificación one-hot //convercion a binarios
# Definir transformadores para características numéricas y categóricas
numeric_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
numeric_transformer = StandardScaler()

categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [ ]:
#12
preprocessor

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

In [62]:
#13
# Selección y justificación de la medida de calidad
print("Selección y justificación de la medida de calidad:")
print("Utilizaremos la precisión (accuaracy) como medida de calidad para evaluar los modelos")

# Algoritmos que serán empleados y estrategia para su ajuste
print("Algoritmos que serán empleados y estrategia para su ajuste:")
print("Utilizaremos Gradient Boosting y CatBoost como algoritmos de clasificación")

Selección y justificación de la medida de calidad:
Utilizaremos la precisión (accuaracy) como medida de calidad para evaluar los modelos
Algoritmos que serán empleados y estrategia para su ajuste:
Utilizaremos Gradient Boosting y CatBoost como algoritmos de clasificación


In [ ]:
#14
# Crear el pipeline de preprocesamiento y modelado con Gradient Boosting
pipeline_gb = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(random_state=42))
])


In [ ]:
#15
# Entrenar y ajustar el modelo de Gradient Boosting
pipeline_gb.fit(X_train, y_train)

In [ ]:
#16
# Crear el pipeline de preprocesamiento y modelado con CatBoost
pipeline_catboost = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', CatBoostClassifier(random_state=42, verbose=False))
])

In [ ]:
##17
# Entrenar y ajustar el modelo de CatBoost
pipeline_catboost.fit(X_train, y_train)

In [ ]:
#18
# Realizar predicciones en los conjuntos de prueba//ajuste
y_pred_gb = pipeline_gb.predict(X_test)
y_pred_catboost = pipeline_catboost.predict(X_test)

#Evaluar la precisión

In [ ]:
#19
# Evaluar la precisión y el informe de clasificación para Gradient Boosting
accuracy_gb = accuracy_score(y_test, y_pred_gb)
report_gb = classification_report(y_test, y_pred_gb)

In [ ]:
#20
# Evaluar la precisión y el informe de clasificación para CatBoost
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
report_catboost = classification_report(y_test, y_pred_catboost)

# Mostrar resultado

In [ ]:
#21
# Mostrar resultados
print("Gradient Boosting Classifier:")
print("Accuracy:", accuracy_gb)
print("Classification Report:\n", report_gb)

print("\nCatBoost Classifier:")
print("Accuracy:", accuracy_catboost)
print("Classification Report:\n", report_catboost)

# Curva de aprendizaje

In [ ]:
#22
# Obtener las curvas de aprendizaje para Gradient Boosting
train_sizes, train_scores, test_scores = learning_curve(pipeline_gb, X_train, y_train, cv=5,scoring='accuracy', n_jobs=-1)
# Graficar las curvas de aprendizaje para Gradient Boosting
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_scores.mean(axis=1), 'r-', label='Training Accuracy')
plt.plot(train_sizes, test_scores.mean(axis=1), 'b-', label='Validation Accuracy')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curves - Gradient Boosting')
plt.legend(loc='best')
plt.grid(True)
plt.show()


In [ ]:
#23
# Obtener las curvas de aprendizaje para CatBoost
train_sizes, train_scores, test_scores = learning_curve(pipeline_catboost, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
# Graficar las curvas de aprendizaje para CatBoost
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_scores.mean(axis=1), 'r-', label='Training Accuracy')
plt.plot(train_sizes, test_scores.mean(axis=1), 'b-', label='Validation Accuracy')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curves - CatBoost')
plt.legend(loc='best')
plt.grid(True)
plt.show()

# Línea Base

In [ ]:
#24
# Algoritmo de línea base (por ejemplo, un clasificador aleatorio)
from sklearn.dummy import DummyClassifier

baseline_model = DummyClassifier(strategy='most_frequent')

baseline_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', baseline_model)
])

# Entrenar el modelo de línea base
baseline_pipeline.fit(X_train, y_train)

# Predicciones del modelo de línea base
baseline_y_pred = baseline_pipeline.predict(X_test)

# Medida de calidad del modelo de línea base
baseline_report = classification_report(y_test, baseline_y_pred)
print(baseline_report)

# Comparación de línea base y resultados propios (Gradient Boosting)
print("Comparación con Gradient Boosting:")
print("Línea base:")
print(baseline_report)
print("Gradient Boosting:")
print(report_gb)

# Comparación de línea base y resultados propios (CatBoost)
print("Comparación con CatBoost:")
print("Línea base:")
print(baseline_report)
print("CatBoost:")
print(report_catboost)